In [1]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D,LSTM,BatchNormalization ,Dense,  Dropout
import tensorflow as tf 

In [2]:
# path to the directory
RAVD = "RAVDESS/Dataset/"
print(RAVD)

RAVDESS/Dataset/


In [3]:
dirl_list = os.listdir(RAVD)
dirl_list.sort()

emotion = []
gender = []
path = []
for i in dirl_list:
    fname = os.listdir(RAVD + i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(RAVD + i + '/' + f)

        
RAVD_df = pd.DataFrame(emotion)
RAVD_df = RAVD_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAVD_df = pd.concat([pd.DataFrame(gender),RAVD_df],axis=1)
RAVD_df.columns = ['gender','emotion']
RAVD_df['labels'] =RAVD_df.gender + '_' + RAVD_df.emotion
RAVD_df['source'] = 'RAVDESS'  
RAVD_df = pd.concat([RAVD_df,pd.DataFrame(path, columns = ['path'])],axis=1)
RAVD_df = RAVD_df.drop(['gender', 'emotion'], axis=1)
RAVD_df.labels.value_counts()

male_neutral       144
female_neutral     144
male_happy          96
male_sad            96
male_angry          96
male_fear           96
male_disgust        96
male_surprise       96
female_happy        96
female_sad          96
female_angry        96
female_fear         96
female_disgust      96
female_surprise     96
Name: labels, dtype: int64

In [4]:
# NOISE
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data
# STRETCH
def stretch(data):
    return librosa.effects.time_stretch(data,rate=0.8)
# SHIFT
def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)
# PITCH
def pitch(data, sampling_rate):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=0.7)

In [5]:
# Trying different functions above
path = np.array(RAVD_df['path'])[471]
data, sample_rate = librosa.load(path)

In [6]:
def feat_ext(data):
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    return mfcc

def get_feat(path):
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    # normal data
    res1 = feat_ext(data)
    result = np.array(res1)
    #data with noise
    noise_data = noise(data)
    res2 = feat_ext(noise_data)
    result = np.vstack((result, res2))
    #data with stretch and pitch
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = feat_ext(data_stretch_pitch)
    result = np.vstack((result, res3))
    return result

In [7]:
RAVD_df.head()

,labels,source,path
0,male_neutral,RAVDESS,RAVDESS/Dataset/Actor_01/03-01-01-01-01-01-01.wav
1,male_neutral,RAVDESS,RAVDESS/Dataset/Actor_01/03-01-01-01-01-02-01.wav
2,male_neutral,RAVDESS,RAVDESS/Dataset/Actor_01/03-01-01-01-02-01-01.wav
3,male_neutral,RAVDESS,RAVDESS/Dataset/Actor_01/03-01-01-01-02-02-01.wav
4,male_neutral,RAVDESS,RAVDESS/Dataset/Actor_01/03-01-02-01-01-01-01.wav


In [8]:
X, Y = [], []
for path, emotion in zip(RAVD_df['path'], RAVD_df['labels']):
    feature = get_feat(path)
    for ele in feature:
        X.append(ele)
        Y.append(emotion)

In [9]:
Emotions = pd.DataFrame(X)
Emotions['labels'] = Y
Emotions.to_csv('emotion.csv', index=False)
Emotions.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,labels
0,-645.880737,72.619637,0.841305,16.399446,10.241591,0.653901,-4.328002,-4.258932,-14.575824,-3.806285,...,-4.042665,2.519602,-4.972247,0.566447,1.035527,-5.416451,-2.449469,-1.782306,-1.212806,male_neutral
1,-464.352938,35.099598,7.859345,10.359874,5.908057,-0.696702,-3.446659,-7.037556,-9.921493,-6.222395,...,-2.087463,-1.622157,-2.390549,-0.620618,0.513910,-3.323046,-3.454055,-0.703648,-3.102154,male_neutral
2,-712.554077,70.982697,-0.730810,16.815830,7.990942,-0.822050,-6.391445,-6.146833,-15.975613,-0.313743,...,-3.398212,2.348455,-6.031178,1.242807,-1.837245,-5.671288,-3.115946,-1.071852,-4.964334,male_neutral
3,-634.663269,72.324066,-2.998578,20.209740,10.674217,-1.151011,-2.813592,-7.923956,-16.231958,-2.627547,...,-4.167823,0.719305,-4.954944,-0.688254,0.114427,-6.475232,-1.695712,-2.899729,0.377745,male_neutral
4,-504.513194,41.542561,5.826739,13.918396,7.391273,-1.736970,-3.448119,-8.976390,-12.569323,-5.130380,...,-2.969334,-2.081280,-3.003049,-0.797526,-0.272498,-4.847971,-3.838261,-0.704348,-1.999350,male_neutral


In [10]:
X = Emotions.iloc[: ,:-1].values
Y = Emotions['labels'].values

In [11]:
# As this is a multiclass classification problem onehotencoding our Y
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [12]:
# Train and Test Split 
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3240, 20), (3240, 14), (1080, 20), (1080, 14))

In [13]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3240, 20), (3240, 14), (1080, 20), (1080, 14))

In [14]:
#CNN
x_traincnn =np.expand_dims(x_train, axis=2)
x_testcnn= np.expand_dims(x_test, axis=2)
x_traincnn.shape, y_train.shape, x_testcnn.shape, y_test.shape

((3240, 20, 1), (3240, 14), (1080, 20, 1), (1080, 14))

In [16]:
#CNN
model = Sequential()
model.add(Conv1D(2048, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(1024, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(LSTM(256, return_sequences=True))

model.add(LSTM(128))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(14, activation='softmax'))

optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 20, 2048)          12288     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 2048)         0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 10, 2048)         8192      
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 10, 1024)          10486784  
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 5, 1024)          0         
 1D)                                                             
                                                        

In [17]:
#CNN
history = model.fit(x_traincnn, y_train, batch_size=64, epochs=150, validation_data=(x_testcnn, y_test))

Epoch 1/150
51/51 [==============================] - 102s 2s/step - loss: 2.5994 - accuracy: 0.1130 - val_loss: 2.6324 - val_accuracy: 0.1194
Epoch 2/150
51/51 [==============================] - 100s 2s/step - loss: 2.5346 - accuracy: 0.1590 - val_loss: 2.6254 - val_accuracy: 0.1111
Epoch 3/150
51/51 [==============================] - 97s 2s/step - loss: 2.4592 - accuracy: 0.1904 - val_loss: 2.6102 - val_accuracy: 0.1491
Epoch 4/150
51/51 [==============================] - 97s 2s/step - loss: 2.3659 - accuracy: 0.2204 - val_loss: 2.6059 - val_accuracy: 0.1324
Epoch 5/150
51/51 [==============================] - 97s 2s/step - loss: 2.2638 - accuracy: 0.2497 - val_loss: 2.5995 - val_accuracy: 0.1037
Epoch 6/150
51/51 [==============================] - 98s 2s/step - loss: 2.1862 - accuracy: 0.2694 - val_loss: 2.5677 - val_accuracy: 0.0963
Epoch 7/150
51/51 [==============================] - 98s 2s/step - loss: 2.0518 - accuracy: 0.3043 - val_loss: 2.4975 - val_accuracy: 0.1787
Epoch 8/150

In [18]:
#CNN
print("Accuracy of our model on test data : " , 
      model.evaluate(x_testcnn,y_test)[1]*100 , "%")

34/34 [==============================] - 1s 34ms/step - loss: 1.3710 - accuracy: 0.8259
Accuracy of our model on test data :  82.59259462356567 %


In [19]:
#CNN
pred_test = model.predict(x_testcnn)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)

34/34 [==============================] - 2s 33ms/step


,Predicted Labels,Actual Labels
0,female_fear,female_fear
1,male_angry,male_angry
2,female_fear,female_fear
3,female_neutral,female_neutral
4,female_surprise,female_angry
5,female_surprise,female_surprise
6,male_sad,male_fear
7,male_happy,male_happy
8,female_fear,female_fear
9,male_sad,male_sad


In [20]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

   female_angry       0.88      0.82      0.85        68
 female_disgust       0.82      0.90      0.86        61
    female_fear       0.86      0.86      0.86        79
   female_happy       0.82      0.72      0.77        75
 female_neutral       0.82      0.97      0.89       120
     female_sad       0.87      0.69      0.77        80
female_surprise       0.87      0.88      0.87        66
     male_angry       0.92      0.89      0.90        64
   male_disgust       0.76      0.70      0.73        74
      male_fear       0.72      0.85      0.78        72
     male_happy       0.76      0.82      0.79        66
   male_neutral       0.86      0.88      0.87       111
       male_sad       0.78      0.75      0.76        71
  male_surprise       0.86      0.75      0.80        73

       accuracy                           0.83      1080
      macro avg       0.83      0.82      0.82      1080
   weighted avg       0.83   

In [21]:
model_name = 'model3.h5'
save_dir = os.path.join('./', 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at ./saved_models\model3.h5 


In [15]:
import json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

NameError: name 'model' is not defined

In [17]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models/model3.h5")
print("Loaded model from disk")

Loaded model from disk


In [18]:
#livedf= pd.DataFrame(columns=['feature'])
# X, sample_rate = librosa.load('RAVDESS/Dataset/Actor_01/03-01-02-02-02-01-01.wav',duration=2.5,sr=22050*2,offset=0.5)
X,Y=[],[]
path="RAVDESS/Dataset/Actor_19/03-01-03-02-01-02-19.wav"
# path="today.wav"
# data, sample_rate = librosa.load(path)
data, sample_rate = librosa.load(path, duration=2.5,sr=22050*2, offset=0.6)
# normal data
res1 = feat_ext(data)
result = np.array(res1)
#data with noise
noise_data = noise(data)
res2 = feat_ext(noise_data)
result = np.vstack((result, res2))
#data with stretch and pitch
new_data = stretch(data)
data_stretch_pitch = pitch(new_data, sample_rate)
res3 = feat_ext(data_stretch_pitch)
result = np.vstack((result, res3))
for ele in result:
    X.append(ele)
    Y.append(emotion)
sample_rate = np.array(sample_rate)
mfcc = np.mean(librosa.feature.mfcc(y=data_stretch_pitch, sr=sample_rate).T, axis=0)
featurelive = mfcc
livedf2 = featurelive

In [19]:
livedf2= pd.DataFrame(data=livedf2)
livedf2 = livedf2.stack().to_frame().T
livedf2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,-574.453918,90.764946,-22.942814,8.266184,-10.9204,8.592998,-7.048609,-4.984391,-25.811499,-8.443547,-12.745316,-2.639219,1.178126,-11.769129,-8.46141,-1.8621,-8.877709,0.573254,-1.561605,-4.329304


In [20]:
twodim= np.expand_dims(livedf2, axis=2)
twodim

array([[[-5.7445392e+02],
        [ 9.0764946e+01],
        [-2.2942814e+01],
        [ 8.2661839e+00],
        [-1.0920400e+01],
        [ 8.5929976e+00],
        [-7.0486093e+00],
        [-4.9843907e+00],
        [-2.5811499e+01],
        [-8.4435472e+00],
        [-1.2745316e+01],
        [-2.6392190e+00],
        [ 1.1781260e+00],
        [-1.1769129e+01],
        [-8.4614096e+00],
        [-1.8620998e+00],
        [-8.8777094e+00],
        [ 5.7325351e-01],
        [-1.5616047e+00],
        [-4.3293042e+00]]], dtype=float32)

In [21]:
livepreds = loaded_model.predict(twodim, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 3s 3s/step


In [22]:
livepreds

array([[9.6242211e-04, 2.6862330e-03, 6.5985769e-01, 2.5770647e-02,
        4.9218708e-03, 4.2449296e-04, 1.1171002e-02, 2.9137545e-05,
        3.6352358e-05, 8.8604327e-05, 2.2834145e-05, 2.0379850e-01,
        9.0152293e-02, 7.7969125e-05]], dtype=float32)

In [23]:
livepreds.shape

(1, 14)

In [24]:
livepredictions = (encoder.inverse_transform((livepreds)))
livepredictions

array([['female_fear']], dtype=object)